In [5]:
import os, pprint, csv, json, gensim, random, pickle
path_data = "/data/cm/"
path = "/home/rovera/cm/"
# prepare training data for word correction. There are 3 types of words: 1) wrong words that contain OCR errors
# 2) correct, but out-of-vocabulary words (which are the most domain-specific) and 3) correct, in-vocabulary words

In [2]:
we = gensim.models.KeyedVectors.load_word2vec_format(path_data+'word_emb/cc.de.300.vec', binary=False)

KeyboardInterrupt: 

In [9]:
ann_set = open(path_data+"data/ocr_correction/complete/annotation_set_1.csv")
ann_set = list(csv.reader(ann_set, delimiter="\t"))
ann_set_2 = open(path_data+"data/ocr_correction/complete/annotation_set_2.csv")
ann_set_2 = list(csv.reader(ann_set_2, delimiter="\t"))

In [ ]:
pages_by_year = json.load(open(path_data+"data/metadata/short_metadata/pages_by_year.json", 'r', encoding='utf-8'))

In [ ]:
sample = []
for k,v in pages_by_year.items():
    decade = v
    random.shuffle(decade)
    if len(decade) >= 3000:
        for el in decade[:3000]:
            sample.append(el)
    else:
        for el in decade:
            sample.append(el)
random.shuffle(sample)

In [ ]:
# collect in-vocabulary words
correct = set()
for filename in sample:
    
    journal = filename.split('_')[0]
    
    file = json.load(open(path_data+"output/nlp_pipeline/tokenized_CM/"+journal+"/"+filename+".json", 'r', encoding="utf-8"))
    
    for token in file['tokens']:
          
        try:
            we[token]
            correct.add(token)
        except KeyError:
            pass
        
    if len(correct) % 100 == 0:
        print(len(correct), end=" ")
    if len(correct) > 50000:
        break
print("Done, collected ", len(correct), "words.")

In [ ]:
correct = list(correct)
pprint.pprint(correct[:100])
correct = [(token, token) for token in correct]
# CAUTION! It overwrites! Do it if you know what you are doing! :) 
#with open(path_data+"data/ocr_correction/training_data/correct.pickle", 'wb') as outfile:
#    pickle.dump(correct, outfile)

In [10]:
wrong = []
oov = []
for a_s in [ann_set, ann_set_2]:
    for line in a_s:
        if line[1] != "" and line[0] != line[1]:
            wrong.append((line[0], line[1]))
        elif line[1] != "" and line[0] == line[1]:
            oov.append((line[0], line[1]))
print(len(wrong))
print(len(oov))

7506
2048


In [11]:
#with open(path_data+"data/ocr_correction/training_data/wrong.pickle", 'wb') as outfile:
#    pickle.dump(wrong, outfile)

In [12]:
#with open(path_data+"data/ocr_correction/training_data/oov_correct.pickle", 'wb') as outfile:
#    pickle.dump(oov, outfile)